In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import spacy
from spacy.matcher import PhraseMatcher

# GOOGLE + FACEBOOK

In [7]:
def get_text_from_csv(fn): # get ad's text from csv file, return a dataframe with text, ad_id,ad_url, and report_url
    df = pd.read_csv(fn)
    df = df[df['ad_type'] == 'Text'].reset_index(drop = True) # only get text ads
    df_url = df[['ad_id','ad_url']].reset_index(drop = True) # get the urls
    urls = df_url['ad_url'].to_list() # create a list so we could get the report_urls
    report_urls = []
    for url in urls:
        entity_id = url.split('/')[-3]
        creative_id = url.split('/')[-1]
        report_url = 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id={}&creative_id={}&hl=en'.format(entity_id,creative_id)
        report_urls.append(report_url)
    df_report = pd.DataFrame(report_urls).rename(columns = {0:'report_url'})
    ad_text = [] # get texts from reports
    for report_url in report_urls:
        response = requests.get(report_url)
        text = response.text.split('"]')[0].split('[')[-1]
        ad_text.append(text)  
    df_text = pd.DataFrame(ad_text).rename(columns = {0:'text'})
    df_new = df[df['ad_type']=='Text'][['ad_id','ad_url']].reset_index(drop = True) 
    df = pd.concat([df_new,df_report,df_text],axis=1)
    return df 

In [8]:
google_df = get_text_from_csv('GoogleAds/NY.csv')
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ad_id       117 non-null    object
 1   ad_url      117 non-null    object
 2   report_url  117 non-null    object
 3   text        117 non-null    object
dtypes: object(4)
memory usage: 3.8+ KB


In [9]:
google_df.head()

,ad_id,ad_url,report_url,text
0,CR145945153381597184,https://transparencyreport.google.com/politica...,https://transparencyreport.google.com/transpar...,"""Representative for New York's First Congressi..."
1,CR66373359440494592,https://transparencyreport.google.com/politica...,https://transparencyreport.google.com/transpar...,"""New Yorkers deserve a governor who works as h..."
2,CR103553482572693504,https://transparencyreport.google.com/politica...,https://transparencyreport.google.com/transpar...,"""\""Help Khaled \"" to fight the three enemies o..."
3,CR474309078336667648,https://transparencyreport.google.com/politica...,https://transparencyreport.google.com/transpar...,"""\""Help Khaled \"" to fight the three enemies o..."
4,CR277557794738012160,https://transparencyreport.google.com/politica...,https://transparencyreport.google.com/transpar...,"""“Simply Put, your vote count to get Free educ..."


In [10]:
# read facebook ads
facebook_df = pd.read_csv('FacebookAds/New York.csv')
# only keep text column
facebook_df.drop(columns = ['Unnamed: 0','byline'],inplace = True)
pd.set_option('display.max_colwidth', None)
facebook_df

text
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [11]:
# cncat google_df and facebook_df by 'text'
df_text = pd.concat([google_df,facebook_df])
df_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6928 entries, 0 to 6810
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ad_id       117 non-null    object
 1   ad_url      117 non-null    object
 2   report_url  117 non-null    object
 3   text        6919 non-null   object
dtypes: object(4)
memory usage: 270.6+ KB


In [12]:
# check the dataset 
if df_text['text'].isnull().sum():
    print (df_text['text'].isnull().sum())
    df_text = df_text[df_text['text'].notnull()]
    df_text.reset_index(drop=True,inplace=True)

9


In [13]:
# function that filter the urls and symbols in the text, and remove duplicated ads
def clean_text(df_text):
    def filter_text(x):
        url = 'http[s]?://\S+'
        x = re.sub(url,'',x)
        x = re.sub("[^\w\s]",' ',x) # filter symbols
        x = re.sub("\s+",' ',x)
        ls=[w.lower() for w in x.split()] 
        return ' '.join(ls)
    df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
    df_text['text'] = df_text['text'].replace('/u0026', ' ')
    df_text.drop_duplicates(subset = 'text',keep = 'first', inplace = True)
    df_text.reset_index(drop = True,inplace = True)
    return df_text

In [14]:
df_text = clean_text(df_text)
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ad_id       65 non-null     object
 1   ad_url      65 non-null     object
 2   report_url  65 non-null     object
 3   text        300 non-null    object
dtypes: object(4)
memory usage: 9.5+ KB


In [15]:
# import the Lexicon
with open ('lexicon.json') as f1:
    dic1 = json.load(f1)
# check the keys
dic1.keys()

dict_keys(['covid', 'economic', 'education', 'environment', 'foreign policy', 'governance', 'health', 'immigration', 'military', 'safety', 'social and cultural', 'social programs', 'donation'])

In [16]:
# import en_core_web
#This only happens to Ying that she couldn't import 'en_core_web_sm' in Jupyter Notebook, so she imports it with the full path
nlp = spacy.load("/usr/local/Cellar/jupyterlab/3.2.9/libexec/lib/python3.9/site-packages/en_core_web_lg/en_core_web_lg-3.2.0/")

In [17]:
# the function that find the lexicon words in the text
def find_words(x,lexicon):
    topics= lexicon.keys()  
    doc = nlp(x) # nlp() is spaCy 2.2 English language model 
    words= []
    for t in topics:
        matcher = PhraseMatcher(nlp.vocab)
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms]
        matcher.add("TerminologyList", patterns) # spaCy2.2 phrase matcher
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    if words:
        words = list(set(words))
        return ','.join(words)
    else:
        return('no words')

In [18]:
# tagging the topic in each message
def find_topic(x,lexicon):
    topics= lexicon.keys()    
    if x=='no words':
        return ''    
    if x != 'no words': 
        words = x.split(',')
        labels = []        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                labels.append(t)                
                #l = sorted(labels)        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)
            

In [19]:
def get_word_lable(df_text):
    df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic1))
    df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic1))
    df_text['m_label'] = df_text['m_label'].apply(lambda x: 'no topic' if x=='' else x)
    return df_text

In [20]:
get_word_lable(df_text)


,ad_id,ad_url,report_url,text,words,m_label
0,CR145945153381597184,https://transparencyreport.google.com/political-ads/library/advertiser/AR435848230316539904/creative/CR145945153381597184,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR435848230316539904&creative_id=CR145945153381597184&hl=en,representative for new york s first congressional district serving you,no words,no topic
1,CR66373359440494592,https://transparencyreport.google.com/political-ads/library/advertiser/AR388565450750951424/creative/CR66373359440494592,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR388565450750951424&creative_id=CR66373359440494592&hl=en,new yorkers deserve a governor who works as hard as they do it s why gov kathy hochul has signed over 400 bills in her first six months,no words,no topic
2,CR103553482572693504,https://transparencyreport.google.com/political-ads/library/advertiser/AR571358124997345280/creative/CR103553482572693504,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR571358124997345280&creative_id=CR103553482572693504&hl=en,help khaled to fight the three enemies of humanity poverty ignorance and disease simply put your vote count to get free education in universities for american students,"american students,free education,enemies,education","education,foreign policy,social programs"
3,CR277557794738012160,https://transparencyreport.google.com/political-ads/library/advertiser/AR571358124997345280/creative/CR277557794738012160,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR571358124997345280&creative_id=CR277557794738012160&hl=en,simply put your vote count to get free education in universities for american students help khaled to fight the three enemies of humanity poverty ignorance and disease,"american students,free education,enemies,education","education,foreign policy,social programs"
4,CR268569836936757248,https://transparencyreport.google.com/political-ads/library/advertiser/AR462572616343879680/creative/CR268569836936757248,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR462572616343879680&creative_id=CR268569836936757248&hl=en,chip in today and help robert fight for our community in washington robert needs your help to stop corporate pacs from buying this seat,chip in,donation
...,...,...,...,...,...,...
295,NaN,NaN,NaN,crime is running rampant in our streets and our communities are suffering in congress you can count on me to spearhead the efforts to fully support law enforcement and help tackle this issue head on,no words,no topic
296,NaN,NaN,NaN,i am grateful to have qualified for the nrcc young guns on the radar list i look forward to helping take back the house,guns,safety
297,NaN,NaN,NaN,westchester county news county executive latimer proves affordable housing is a top priority in westchester install news app trusted by millions to stay informed of latest westchester county local news,affordable housing,social programs
298,NaN,NaN,NaN,nassau county news gas tax cut in nassau county after minority caucus push install news app trusted by millions to stay informed of latest nassau county local news,tax,economic


In [21]:
df_text.to_csv('ad_ny.csv')

In [22]:
def count_topic(df_text):
    df_tag = df_text['m_label'].value_counts().rename_axis('topics').reset_index(name='counts')
    df_tag = df_tag.assign(single_topic=df_tag['topics'].str.split(',')).explode('single_topic').reset_index(drop = True)
    df_tag = df_tag.groupby('single_topic').sum().reset_index().sort_values(by = 'counts', ascending = False)
    df_tag.reset_index(drop = True, inplace = True)
    return df_tag

In [23]:
text_topic = count_topic(df_text)
text_topic

,single_topic,counts
0,no topic,124
1,economic,42
2,social and cultural,35
3,environment,33
4,health,32
5,education,26
6,donation,25
7,social programs,24
8,safety,22
9,immigration,16


In [24]:
def count_word(df_text):
    df_words = df_text['words'].value_counts().rename_axis('words').reset_index(name='counts')
    df_words = df_words.assign(single_word=df_words['words'].str.split(',')).explode('single_word').reset_index(drop = True)
    df_words = df_words.groupby('single_word').sum().reset_index().sort_values(by = 'counts', ascending = False)
    df_words.reset_index(drop = True, inplace = True)
    return df_words

In [25]:
text_word = count_word(df_text)
text_word

,single_word,counts
0,no words,124
1,health,24
2,education,17
3,climate,17
4,donate,13
...,...,...
193,living conditions,1
194,liberate,1
195,lebanon,1
196,latino,1


# GOOGLE

In [26]:
if google_df['text'].isnull().sum():
    print (google_df['text'].isnull().sum())
    google_df = google_df[google_df['text'].notnull()]
    google_df.reset_index(drop=True,inplace=True)

In [27]:
google_df = clean_text(google_df)
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ad_id       65 non-null     object
 1   ad_url      65 non-null     object
 2   report_url  65 non-null     object
 3   text        65 non-null     object
dtypes: object(4)
memory usage: 2.2+ KB


In [28]:
google_df = get_word_lable(google_df)
google_df.head()

,ad_id,ad_url,report_url,text,words,m_label
0,CR145945153381597184,https://transparencyreport.google.com/political-ads/library/advertiser/AR435848230316539904/creative/CR145945153381597184,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR435848230316539904&creative_id=CR145945153381597184&hl=en,representative for new york s first congressional district serving you,no words,no topic
1,CR66373359440494592,https://transparencyreport.google.com/political-ads/library/advertiser/AR388565450750951424/creative/CR66373359440494592,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR388565450750951424&creative_id=CR66373359440494592&hl=en,new yorkers deserve a governor who works as hard as they do it s why gov kathy hochul has signed over 400 bills in her first six months,no words,no topic
2,CR103553482572693504,https://transparencyreport.google.com/political-ads/library/advertiser/AR571358124997345280/creative/CR103553482572693504,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR571358124997345280&creative_id=CR103553482572693504&hl=en,help khaled to fight the three enemies of humanity poverty ignorance and disease simply put your vote count to get free education in universities for american students,"american students,free education,enemies,education","education,foreign policy,social programs"
3,CR277557794738012160,https://transparencyreport.google.com/political-ads/library/advertiser/AR571358124997345280/creative/CR277557794738012160,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR571358124997345280&creative_id=CR277557794738012160&hl=en,simply put your vote count to get free education in universities for american students help khaled to fight the three enemies of humanity poverty ignorance and disease,"american students,free education,enemies,education","education,foreign policy,social programs"
4,CR268569836936757248,https://transparencyreport.google.com/political-ads/library/advertiser/AR462572616343879680/creative/CR268569836936757248,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR462572616343879680&creative_id=CR268569836936757248&hl=en,chip in today and help robert fight for our community in washington robert needs your help to stop corporate pacs from buying this seat,chip in,donation


In [29]:
google_topic = count_topic(google_df)
google_topic

,single_topic,counts
0,no topic,33
1,donation,13
2,economic,8
3,safety,4
4,governance,3
5,education,2
6,environment,2
7,foreign policy,2
8,social programs,2


In [30]:
google_word = count_word(google_df)
google_word

,single_word,counts
0,no words,33
1,middle class,7
2,chip in,6
3,donate,6
4,taxes,5
5,gun violence,3
6,gun,3
7,filibuster,3
8,bail,2
9,free education,2


# FACEBOOK

In [31]:
if facebook_df['text'].isnull().sum():
    print (facebook_df['text'].isnull().sum())
    facebook_df = facebook_df[facebook_df['text'].notnull()]
    facebook_df.reset_index(drop=True,inplace=True)

9


In [32]:
facebook_df = clean_text(facebook_df)
facebook_df.info

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_75198/222855393.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_75198/222855393.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].replace('/u0026', ' ')
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_75198/222855393.py:12: SettingWithCopyWarning: 
A value is try

<bound method DataFrame.info of                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [33]:
facebook_df = get_word_lable(facebook_df)
facebook_df.head()

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_75198/1334858953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic1))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_75198/1334858953.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic1))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_75198/1334858953.py:4: SettingWithCopyWa

,text,words,m_label
0,over the past two years we have witnessed how covid 19 deepened the financial need of organizations in our community we depend on while highlighting gaps that existed before the public health emergency in march president biden signed a government funding law that included 21 9 million for projects throughout brooklyn i fought for this money will help provide food for the hungry fund community healthcare initiatives support organizations working to uplift our neighborhoods through youth development and empowerment deepen our cultural understanding and more to learn more about these projects and to keep up to date with my work both in new york and washington be sure to subscribe to my communications,"health,healthcare,covid 19,covid","covid,health"
1,over the last 25 years america has virtually eliminated plane crashes before that we were on track to see one passenger jet crash per week so what changed our new video explains,no words,no topic
2,𝗬𝗢𝗨 𝗥𝗘 𝗜𝗡𝗩𝗜𝗧𝗘𝗗 𝘁𝗼 𝗮 𝗳𝗿𝗲𝗲 𝗹𝗶𝘃𝗲𝘀𝘁𝗿𝗲𝗮𝗺 𝗲𝘃𝗲𝗻𝘁 𝗼𝗻 𝘁𝗵𝗲 𝗳𝘂𝘁𝘂𝗿𝗲 𝗼𝗳 𝘁𝗵𝗲 𝗰𝗹𝗶𝗺𝗮𝘁𝗲 𝗺𝗼𝘃𝗲𝗺𝗲𝗻𝘁 𝗠𝗮𝘆 𝟯 𝟭 𝟮 𝟯𝟬 𝗽𝗺 𝗘𝗦𝗧 new york governor kathy hochul joins north carolina governor roy cooper and u s department of energy secretary jennifer granholm to talk about what the federal government and state legislatures are doing to combat climate change right now,"climate,climate change",environment
3,what s next in the climate movement find out at a livestream event with new york governor kathy hochul north carolina governor roy cooper and energy secretary jennifer granholm on may 3 1 2 30 pm est welcome remarks from nrdc president manish bapna,climate,environment
4,we re at a critical turning point in the future of our climate join new york governor kathy hochul at a special livestream event to find out how states and the federal government are taking action on climate tuesday may 3 1 2 30 pm est,climate,environment


In [34]:
facebook_topic = count_topic(facebook_df)
facebook_topic

,single_topic,counts
0,no topic,91
1,social and cultural,35
2,economic,34
3,health,32
4,environment,31
5,education,24
6,social programs,22
7,safety,18
8,immigration,16
9,donation,12


In [35]:
facebook_word = count_word(facebook_df)
facebook_word

,single_word,counts
0,no words,91
1,health,24
2,climate,16
3,education,15
4,school,11
...,...,...
187,health coverage,1
188,gun violence,1
189,green jobs,1
190,great teachers,1
